# House Prices - Advanced Regression Techniques (Kaggle)

Run after following notebooks are run:
1. 01 Data Cleaning
2. 02 Exploratory Data Analysis

#### Development Ideas
*  Need to scale discrete numerical variables as well

In [1]:
### set up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import seaborn as sns

pd.options.display.max_rows = 100

## Load Data

In [2]:
### load data (cleaned)
train = pd.read_csv('train_clean_truncated.csv')
test = pd.read_csv('test_clean.csv')

### convert all object datatypes to category
for col in train.select_dtypes(include='object'):
    train[col] = train[col].astype('category')

for col in test.select_dtypes(include='object'):
    test[col] = test[col].astype('category')

### convert additional columns to category datatype
train['MSSubClass'] = train['MSSubClass'].astype('category')
test['MSSubClass'] = test['MSSubClass'].astype('category')

train.shape, test.shape

((1460, 67), (1459, 80))

In [3]:
## separate in to predictor and response tables (x and y)
x_train = train.drop('SalePrice', axis=1)
y_train = train.SalePrice

## for the test data, take only the x vars present in the train data,
x_train_cols = x_train.columns.to_list()
x_test = test[x_train_cols]

## One Hot Encoding

In [4]:
x_train_onehot = pd.get_dummies(x_train, drop_first=True)
x_test_onehot = pd.get_dummies(x_test, drop_first=True)
print(x_train_onehot.shape, x_test_onehot.shape)
x_train_onehot.columns.to_list()

(1460, 200) (1459, 196)


['LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'MoSold',
 'YrSold',
 'age_sold',
 'age_remodel_sold',
 'age_garage_sold',
 'bsmt_bath',
 'bath',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90',
 'MSSubClass_120',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Alley_Pave',
 'LotShape_Reg',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'LotConfig_CulDSac',
 'LotConfig_FR',
 'LotConfig_Inside',
 'LandSlope_MOrS',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_C

In [5]:
## add on missing columns to train data due to onehot encoding
train_cols_to_add = [var for var in x_test_onehot.columns.to_list() if var not in x_train_onehot.columns.to_list()]
for var in train_cols_to_add:
    x_train_onehot[var] = False

## add on missing columns to test data due to onehot encoding
test_cols_to_add = [var for var in x_train_onehot.columns.to_list() if var not in x_test_onehot.columns.to_list()]
for var in test_cols_to_add:
    x_test_onehot[var] = False

print(x_train_onehot.shape, x_test_onehot.shape)

(1460, 201) (1459, 201)


## Modelling - Cross-Validation and Optimisation

### Random Forest

In [6]:
## hyperparameter tuning: n_estimators
param_grid = {'n_estimators' : np.arange(160,210,20)}
kf = KFold(n_splits=6, shuffle=True, random_state=8)

rf_tuning = GridSearchCV(RandomForestRegressor(random_state=8), param_grid, scoring='neg_mean_squared_error', cv=kf)
rf_tuning.fit(x_train_onehot.values, y_train.values)
rf_tuning_params = rf_tuning.cv_results_['param_n_estimators']
rf_tuning_mse = -rf_tuning.cv_results_['mean_test_score']

## find best param with lowest mse, and plot params against mse
print(rf_tuning.best_params_)
plt.plot(rf_tuning_params, rf_tuning_mse)
plt.xlabel('N_estimators')
plt.ylabel('Mean Squared Error')
None

In [ ]:
## instantiate rf algorithm, using best tuned param
rf = RandomForestRegressor(n_estimators=180, random_state=8)

## cross validation
kf = KFold(n_splits=6, shuffle=True, random_state=8)
rf_cv_scores = cross_val_score(rf, x_train_onehot.values, y_train.values, scoring='neg_mean_squared_error', cv=kf)
print(f'avg MSE: {-np.mean(rf_cv_scores)}\n', f'splits MSE: {-rf_cv_scores}')

In [ ]:
## List feature importance
rf.fit(x_train_onehot.values, y_train.values)
rf_ft_imp = pd.DataFrame(data={'Features': x_train_onehot.columns, 'Gini Importance':rf.feature_importances_*100})
rf_ft_imp.sort_values(by='Gini Importance', ascending=False).head(15)

### KNN

In [ ]:
## limit x vars to top 10 by gini importance (determined in rf section)
top10_vars = rf_ft_imp.sort_values(by='Gini Importance', ascending=False).head(10)['Features'].to_list()
x_train_onehot_top10 = x_train_onehot[top10_vars]

## hyperparameter tuning: alpha
param_grid = {'n_neighbors' : np.arange(3,11)}
kf = KFold(n_splits=6, shuffle=True, random_state=8)

knn_tuning = GridSearchCV(KNeighborsRegressor(weights='distance'), param_grid, scoring='neg_mean_squared_error', cv=kf)
knn_tuning.fit(x_train_onehot_top10.values, y_train.values)
knn_tuning_params = knn_tuning.cv_results_['param_n_neighbors']
knn_tuning_mse = -knn_tuning.cv_results_['mean_test_score']

## find best param with lowest mse, and plot params against mse
print(knn_tuning.best_params_)
plt.plot(knn_tuning_params, knn_tuning_mse)
plt.xlabel('N_neighbors')
plt.ylabel('Mean Squared Error')
None

In [ ]:
## instantiate knn algorithm, using best tuned param
knn = KNeighborsRegressor(n_neighbors=8, weights='distance')

## cross validation
kf = KFold(n_splits=6, shuffle=True, random_state=8)
knn_cv_scores = cross_val_score(knn, x_train_onehot_top10.values, y_train.values, scoring='neg_mean_squared_error', cv=kf)
print(f'avg MSE: {-np.mean(knn_cv_scores)}\n', f'splits MSE: {-knn_cv_scores}')

### Lasso

In [ ]:
## hyperparameter tuning: alpha
param_grid = {'alpha': np.linspace(0.0001, 0.0004, 20)}
kf = KFold(n_splits=5, shuffle=True, random_state=8)

lasso_tuning = GridSearchCV(Lasso(random_state=8), param_grid, scoring='neg_mean_squared_error', cv=kf)
lasso_tuning.fit(x_train_onehot.values, y_train.values)
lasso_tuning_params = lasso_tuning.cv_results_['param_alpha']
lasso_tuning_mse = -lasso_tuning.cv_results_['mean_test_score']

## find best param with lowest mse, and plot params against mse
print(lasso_tuning.best_params_)
plt.plot(lasso_tuning_params, lasso_tuning_mse)
plt.xlabel('Alpha')
plt.ylabel('Mean Squared Error')
None

In [ ]:
## instantiate rf algorith, using best tuned param
lasso = Lasso(alpha=0.000321)

## cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=8)
lasso_cv_scores = cross_val_score(lasso, x_train_onehot.values, y_train.values, scoring='neg_mean_squared_error', cv=kf)
print(f'avg MSE: {np.mean(-lasso_cv_scores)}\n', f'splits MSE: {-lasso_cv_scores}')

## Elastic Net

* Did't follow through with Elastic Net as it would give similar results to lasso (since optimal l1_ratio ~1)

In [ ]:
## hyperparameter tuning: alpha
param_grid = {'alpha': np.linspace(0.0001, 0.0003, 20), 'l1_ratio': [0.7, 0.8, 0.9]}
kf = KFold(n_splits=6, shuffle=True, random_state=8)

elnet_tuning = GridSearchCV(ElasticNet(random_state=8), param_grid, scoring='neg_mean_squared_error', cv=kf)
elnet_tuning.fit(x_train_onehot.values, y_train.values)

## find best param with lowest mse, and plot params against mse
print(elnet_tuning.best_params_, -elnet_tuning.best_score_)

## Test Prediction

In [ ]:
## fit all above models
rf.fit(x_train_onehot.values, y_train.values)
knn.fit(x_train_onehot.values, y_train.values)
lasso.fit(x_train_onehot.values, y_train.values)
None

In [ ]:
## rf prediction
y_testpred_rf = rf.predict(x_test_onehot.values)
y_testpred_rf = np.exp(y_testpred_rf)

## knn prediction
y_testpred_knn = knn.predict(x_test_onehot.values)
y_testpred_knn = np.exp(y_testpred_knn)

## lasso prediction
y_testpred_lasso = lasso.predict(x_test_onehot.values)
y_testpred_lasso = np.exp(y_testpred_lasso)

## rf x lasso stacked average prediction
y_testpred_stacked_rflasso = (y_testpred_rf + y_testpred_lasso)/2

### Save Predictions

In [ ]:
## save in submission format - i.e. Id & SalePrice
Id = np.arange(len(train)+1, len(train)+len(test)+1)

y_testpred_rf = pd.DataFrame(data={'Id':Id, 'SalePrice':y_testpred_rf})
y_testpred_knn = pd.DataFrame(data={'Id':Id, 'SalePrice':y_testpred_knn})
y_testpred_lasso = pd.DataFrame(data={'Id':Id, 'SalePrice':y_testpred_lasso})
y_testpred_stacked_rflasso = pd.DataFrame(data={'Id':Id, 'SalePrice':y_testpred_stacked_rflasso})

## save a csv
y_testpred_rf.to_csv('Submissions/test_pred_rf_v3.csv', index=False)
y_testpred_knn.to_csv('Submissions/test_pred_knn_v2.csv', index=False)
y_testpred_lasso.to_csv('Submissions/test_pred_lasso_v2.csv', index=False)
y_testpred_stacked_rflasso.to_csv('Submissions/test_pred_stacked_v2.csv', index=False)